In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.9 MB/s eta 0

In [3]:
import logging
import pandas as pd
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs
import string
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'simpletransformers'

In [3]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv("/content/drive/MyDrive/train.csv")
df.drop(['Context'], axis = 1, inplace = True)
df

,Response
0,"If everyone thinks you're worthless, then mayb..."
1,"Hello, and thank you for your question and see..."
2,First thing I'd suggest is getting the sleep y...
3,Therapy is essential for those that are feelin...
4,I first want to let you know that you are not ...
...,...
3507,Absolutely not! It is never in a child's best ...
3508,I'm sorry you have tension between you and you...
3509,"The true answer is, ""no one can really say wit..."
3510,How do you help yourself to believe you requir...


In [6]:
df = df.dropna()
df = df.drop_duplicates()

In [7]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)
new_data = {'input_text': [], 'target_text': []}

for index, row in df.iterrows():
    input_text = remove_punctuation(row['Response'])  # Remove punctuation for input
    target_text = row['Response']
    new_data['input_text'].append(input_text)
    new_data['target_text'].append(target_text)

new_df = pd.DataFrame(new_data)

In [8]:
new_df

,input_text,target_text
0,If everyone thinks youre worthless then maybe ...,"If everyone thinks you're worthless, then mayb..."
1,Hello and thank you for your question and seek...,"Hello, and thank you for your question and see..."
2,First thing Id suggest is getting the sleep yo...,First thing I'd suggest is getting the sleep y...
3,Therapy is essential for those that are feelin...,Therapy is essential for those that are feelin...
4,I first want to let you know that you are not ...,I first want to let you know that you are not ...
...,...,...
2474,Being in this position is tough If seeking ano...,Being in this position is tough. If seeking an...
2475,It sounds like you are on the right track Reco...,It sounds like you are on the right track. Rec...
2476,HelloIt sounds like you are really concerned a...,Hello.It sounds like you are really concerned ...
2477,Hi This is an excellent question I think t...,Hi. This is an excellent question ! I think...


In [9]:
new_df.rename(columns={'train_X':'input_text','train_y':'target_text'},inplace=True)
train_df,eval_df=train_test_split(new_df,test_size=0.1)

In [10]:
model_args = Seq2SeqArgs()
model_args.num_train_epochs = 5
model_args.evaluate_generated_text = True
model_args.evaluate_during_training = True
model_args.evaluate_during_training_verbose = True
model_args.overwrite_output_dir=True
model_args.max_length=300

In [11]:
model = Seq2SeqModel(
    encoder_decoder_type="bart",
    encoder_decoder_name="facebook/bart-large",
    args=model_args,

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [12]:
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/2231 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/279 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/248 [00:00<?, ?it/s]

...

Running Epoch 4 of 5:   0%|          | 0/279 [00:00<?, ?it/s]

  0%|          | 0/248 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/31 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarnin

(1395,
 {'global_step': [279, 558, 837, 1116, 1395],
  'eval_loss': [0.1258717607105932,
   0.1018017089414981,
   0.10397279238508593,
   0.11712027677605229,
   0.11040538609508545],
  'train_loss': [0.1355675905942917,
   0.13635686039924622,
   0.08306480199098587,
   0.03101554699242115,
   0.039286643266677856]})

In [13]:
result = model.eval_model(eval_df)

  0%|          | 0/248 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/31 [00:00<?, ?it/s]

Generating outputs:   0%|          | 0/31 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarnin

In [14]:
eval_predict=model.predict(list(eval_df['input_text'].values))

Generating outputs:   0%|          | 0/31 [00:00<?, ?it/s]

In [22]:
eval_df['input_text'][2392]

'Everyone has different experiences going to therapy Being nervous can be a typical emotion one might feel Emotions are our bodys way of telling us important information about ourselves I would suggest talking openly in your sessions about this That way you can process your thoughts and feelings with the guidance of your counselor There are probably underlining emotions fears or insecurities that are being stirred up during your therapy sessions Your counselor might suggest individual counseling depending on what you learn about your anxiety In individual therapy you would have time to deal with your own stressors As you address your issues then you will have tools and skills that will be useful in addressing the couple relationship\xa0'

In [23]:
eval_predict[:1]

["Everyone has different experiences going to therapy. Being nervous can be a typical emotion one might feel. Emotions are our body's way of telling us important information about ourselves. I would suggest talking openly in your sessions about this. That way you can process your thoughts and feelings with the guidance of your counselor. There are probably underlining emotions, fears, or insecurities that are being stirred up during your therapy sessions. Your counselor might suggest individual counseling depending on what you learn about your anxiety. In individual therapy you would have time to deal with your own stressors. As you address your issues then you will have tools and skills that will be useful in addressing the"]

In [4]:
test_df = pd.read_csv("/content/drive/MyDrive/testPunc.csv")

In [5]:
print(test_df)

                                             Testing
0               I purchased cheese ketchup and maida
1                     take the dog for a walk please
2                           i ll be there at 3:30 pm
3             i have a doctors appointment on monday
4  now i ve reached the years the good book allow...
5                                where are you going


In [25]:
to_test=test_df['Testing'].values.tolist()

In [26]:
model.predict(to_test)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `None` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:418: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:437: UserWarnin

['I purchased cheese, ketchup and maida.',
 'take the dog for a walk please!',
 "i'll be there at 3:30 pm.",
 'i have a doctors appointment on monday.',
 'now i ve reached the years the good book allows to man in his lifetime on this earth.',
 'where are you going?']